In [2]:
import numpy as np
import pandas as pd
from ..schemas import DataCatalog
from ..constants import 

SyntaxError: invalid syntax (2655615332.py, line 4)

In [2]:


data_catalog = DataCatalog.load_from_yaml("data_catalog.yaml")

In [3]:
appointments_df = data_catalog.single_data_sources[-1].load()

appointments_df = (
    appointments_df # select only the columns we need, and remove unknown
    .loc[(appointments_df['SUB_ICB_LOCATION_CODE'].isin(const.SUB_ICB_CODES.keys())) & (appointments_df['ACTUAL_DURATION'] != "Unknown / Data Quality")]
    .assign(
        Alliance=appointments_df['SUB_ICB_LOCATION_CODE'].map(
            const.SUB_ICB_CODES), # add the alliance column
        Date=pd.to_datetime(appointments_df['Appointment_Date'], format='%d%b%Y')) # convert the date column to datetime
        # drop the columns we don't need
    .drop(columns=['SUB_ICB_LOCATION_CODE', 'SUB_ICB_LOCATION_ONS_CODE', 'SUB_ICB_LOCATION_NAME', 'ICB_ONS_CODE', 'REGION_ONS_CODE', 'Appointment_Date']))


In [4]:
appointments_binned_df = (appointments_df
                          .groupby(['Alliance',pd.Grouper(key='Date',freq='BA-MAR',label='right'),'ACTUAL_DURATION'])
                          .sum()
                          .reset_index()
                          .rename(columns={'Date':'FY'}).replace({"1-5":"01-05","6-10":"06-10"}))

In [6]:
np.random.uniform(low=0.0, high=1.0, size=100)

array([0.43537468, 0.03624237, 0.15064856, 0.47267519, 0.12851683,
       0.91040526, 0.16088713, 0.08255789, 0.81974263, 0.87500023,
       0.59457152, 0.57844956, 0.43358122, 0.04077597, 0.57132724,
       0.1480304 , 0.99021018, 0.77186945, 0.80901404, 0.11880985,
       0.42336895, 0.0751471 , 0.69858872, 0.5513931 , 0.39472431,
       0.59574685, 0.4675436 , 0.85090586, 0.36487139, 0.40626788,
       0.53728341, 0.32057532, 0.72268878, 0.46937047, 0.9314151 ,
       0.90719237, 0.43297593, 0.29708878, 0.47095081, 0.34919065,
       0.98769113, 0.01856184, 0.39253885, 0.12025316, 0.75834377,
       0.05240629, 0.71941517, 0.08318037, 0.3386926 , 0.65723094,
       0.6118134 , 0.80883695, 0.57880593, 0.364962  , 0.10980673,
       0.50143328, 0.63257975, 0.36869719, 0.06306442, 0.81101938,
       0.46040184, 0.67791748, 0.87390107, 0.0571514 , 0.80329688,
       0.55264029, 0.72453566, 0.01165812, 0.64763558, 0.78228791,
       0.19713836, 0.32501288, 0.22453662, 0.44950226, 0.05446

In [ ]:
appts_dict = appointments_binned_df.groupby('ACTUAL_DURATION')['COUNT_OF_APPOINTMENTS'].sum(numeric_only=True).to_dict()

# calculate the upper bin
upper_bin = max([int(x.split('-')[1][:2]) for x in appts_dict.keys()])


lower_bin_edges_count = dict(sorted({int(k.split("-")[0]):v for k,v in  appts_dict.items()}.items(), key=lambda x:x[0]))

In [ ]:
lower_bin_edges_count

In [ ]:


import numpy as np
from scipy.stats import binned_statistic

# Define your binned data
bins_edges = list(lower_bin_edges_count) + [upper_bin]
values = list(lower_bin_edges_count.values())

# Convert your binned data to numerical values

x = np.repeat(bins_edges, np.diff(np.append([0], bins_edges)).cumsum())


In [ ]:
len(bins_edges), len(values)

In [ ]:

# Compute the probability distribution function
pdf, _, _ = binned_statistic(values, statistic='sum', bins=bins_edges)

# Normalize the probability distribution function
pdf /= pdf.sum()
0
print(pdf)
